In [0]:
from delta import *
from pyspark.sql.types import *

In [0]:
# Step 2: Move the file from the local file system to DBFS
dbutils.fs.rm("dbfs:/temp/products.csv")
dbutils.fs.rm("dbfs:/dim_product/", recurse=True)


True

In [0]:
# dbutils.fs.ls("dbfs:/product/")

In [0]:
%sql
SHOW DATABASES;

databaseName
default
retail_db


In [0]:
%sql
DROP TABLE IF EXISTS retail_db.dimProduct;
DROP SCHEMA IF EXISTS retail_db CASCADE;
CREATE DATABASE IF NOT EXISTS retail_db;
SHOW DATABASES;

databaseName
default
retail_db


In [0]:
delta_table_path = "/dim_product"
dbutils.fs.rm(delta_table_path, recurse=True)

False

In [0]:
# spark.sql("CREATE OR REPLACE TABLE retail_db.dimProduct (ProductID STRING, ProductName STRING, Category STRING,ListPrice STRING) USING DELTA LOCATION '{0}'".format(delta_table_path))
spark.sql("CREATE OR REPLACE TABLE retail_db.dimProduct (ProductID INTEGER, ProductName STRING, Category STRING,ListPrice DOUBLE) USING DELTA LOCATION '{0}'".format(delta_table_path))

DataFrame[]

In [0]:
%sql
USE retail_db;
SHOW TABLES;

database,tableName,isTemporary
retail_db,dimproduct,false


In [0]:
%sql
DESCRIBE EXTENDED retail_db.dimProduct;

col_name,data_type,comment
ProductID,int,null
ProductName,string,null
Category,string,null
ListPrice,double,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,retail_db,
Table,dimProduct,
Created Time,Sun Jul 07 13:08:53 UTC 2024,


In [0]:
!wget https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/products.csv

--2024-07-07 13:08:55--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/products.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14372 (14K) [text/plain]
Saving to: ‘products.csv’

products.csv        100%[===================>]  14.04K  --.-KB/s    in 0.001s  

2024-07-07 13:08:55 (22.0 MB/s) - ‘products.csv’ saved [14372/14372]



In [0]:
!cat products.csv | head -n 10

ProductID,ProductName,Category,ListPrice
771,"Mountain-100 Silver, 38",Mountain Bikes,3399.9900
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.9900
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.9900
774,"Mountain-100 Silver, 48",Mountain Bikes,3399.9900
775,"Mountain-100 Black, 38",Mountain Bikes,3374.9900
776,"Mountain-100 Black, 42",Mountain Bikes,3374.9900
777,"Mountain-100 Black, 44",Mountain Bikes,3374.9900
778,"Mountain-100 Black, 48",Mountain Bikes,3374.9900
779,"Mountain-200 Silver, 38",Mountain Bikes,2319.9900


In [0]:
# Step 2: Move the file from the local file system to DBFS
dbutils.fs.mv("file:/databricks/driver/products.csv", "dbfs:/tmp/products.csv")

True

In [0]:
dbutils.fs.ls("dbfs:/tmp")

[FileInfo(path='dbfs:/tmp/data/', name='data/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/tmp/delta_lab/', name='delta_lab/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/tmp/fireServiceParquet/', name='fireServiceParquet/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/tmp/products.csv', name='products.csv', size=14372, modificationTime=1720357737000),
 FileInfo(path='dbfs:/tmp/spark_lab/', name='spark_lab/', size=0, modificationTime=0)]

In [0]:
productSchema = StructType([
     StructField("ProductID", IntegerType()),
     StructField("ProductName", StringType()),
     StructField("Category", StringType()),
     StructField("ListPrice", DoubleType())
])
productSchema

StructType([StructField('ProductID', IntegerType(), True), StructField('ProductName', StringType(), True), StructField('Category', StringType(), True), StructField('ListPrice', DoubleType(), True)])

In [0]:
# Read the CSV file with multiple options
# df = spark.read.format("csv").options(header=True, delimiter=",", schema=productSchema).csv("dbfs:/tmp/products.csv")
df = spark.read.format("csv").options(header=True, delimiter=",", inferSchema=True).csv("dbfs:/tmp/products.csv")

# Show the DataFrame
df.show(10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|  3399.99|
|      772|Mountain-100 Silv...|Mountain Bikes|  3399.99|
|      773|Mountain-100 Silv...|Mountain Bikes|  3399.99|
|      774|Mountain-100 Silv...|Mountain Bikes|  3399.99|
|      775|Mountain-100 Blac...|Mountain Bikes|  3374.99|
|      776|Mountain-100 Blac...|Mountain Bikes|  3374.99|
|      777|Mountain-100 Blac...|Mountain Bikes|  3374.99|
|      778|Mountain-100 Blac...|Mountain Bikes|  3374.99|
|      779|Mountain-200 Silv...|Mountain Bikes|  2319.99|
|      780|Mountain-200 Silv...|Mountain Bikes|  2319.99|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [0]:
df.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- ListPrice: double (nullable = true)



In [0]:
# df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(delta_table_path)
df.write.format("delta").mode("overwrite").save(delta_table_path)
deltaTable = DeltaTable.forPath(spark, delta_table_path)
deltaTable.toDF().show(10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|  3399.99|
|      772|Mountain-100 Silv...|Mountain Bikes|  3399.99|
|      773|Mountain-100 Silv...|Mountain Bikes|  3399.99|
|      774|Mountain-100 Silv...|Mountain Bikes|  3399.99|
|      775|Mountain-100 Blac...|Mountain Bikes|  3374.99|
|      776|Mountain-100 Blac...|Mountain Bikes|  3374.99|
|      777|Mountain-100 Blac...|Mountain Bikes|  3374.99|
|      778|Mountain-100 Blac...|Mountain Bikes|  3374.99|
|      779|Mountain-200 Silv...|Mountain Bikes|  2319.99|
|      780|Mountain-200 Silv...|Mountain Bikes|  2319.99|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [0]:
%sql
USE retail_db;
SHOW TABLES;

database,tableName,isTemporary
retail_db,dimproduct,false


In [0]:
%sql
SELECT * FROM retail_db.dimproduct LIMIT 10;

ProductID,ProductName,Category,ListPrice
771,"Mountain-100 Silver, 38",Mountain Bikes,3399.99
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.99
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.99
774,"Mountain-100 Silver, 48",Mountain Bikes,3399.99
775,"Mountain-100 Black, 38",Mountain Bikes,3374.99
776,"Mountain-100 Black, 42",Mountain Bikes,3374.99
777,"Mountain-100 Black, 44",Mountain Bikes,3374.99
778,"Mountain-100 Black, 48",Mountain Bikes,3374.99
779,"Mountain-200 Silver, 38",Mountain Bikes,2319.99
780,"Mountain-200 Silver, 42",Mountain Bikes,2319.99


In [0]:
%sql
DESCRIBE EXTENDED retail_db.dimproduct;

col_name,data_type,comment
ProductID,int,null
ProductName,string,null
Category,string,null
ListPrice,double,null
,,
# Delta Statistics Columns,,
Column Names,"ProductID, ProductName, Category, ListPrice",
Column Selection Method,first-32,
,,
# Detailed Table Information,,
